This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = '' #fill This

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests



In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
#https://www.quandl.com/api/v3/datasets/{database_code}/{dataset_code}/data.{return_format}

DB_code='FSE' #database code
DS_code='AFX_X' #data set code
SD='2017-02-23' #startDate
ED='2017-02-23'#endDate
url='https://www.quandl.com/api/v3/datasets/{}/{}/data.{}?start_date={}&end_date={}&api_key={}'\
    .format(DB_code,DS_code,'json',SD,ED,API_KEY)



In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
r=requests.get(url)
print(r.json())

{'dataset_data': {'limit': None, 'transform': None, 'column_index': None, 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'start_date': '2017-02-23', 'end_date': '2017-02-23', 'frequency': 'daily', 'data': [['2017-02-23', 39.72, 39.98, 39.38, 39.79, None, 81945.0, 3260642.0, None, None, None]], 'collapse': None, 'order': None}}


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

## Question 1:
1.Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017

In [5]:
DB_code='FSE' #database code
DS_code='AFX_X' #data set code
SD='2017-01-01' #startDate 
ED='2017-12-31'#endDate End Date is included in returned data
url='https://www.quandl.com/api/v3/datasets/{}/{}/data.{}?start_date={}&end_date={}&api_key={}'\
    .format(DB_code,DS_code,'json',SD,ED,API_KEY)

r=requests.get(url)

## Question 2:
2.Convert the returned JSON object into a Python dictionary.

In [6]:
DictData=r.json()['dataset_data']
assert type(DictData)==dict
print(DictData.keys())

dict_keys(['limit', 'transform', 'column_index', 'column_names', 'start_date', 'end_date', 'frequency', 'data', 'collapse', 'order'])


In [7]:
#print keys for columns names in data.
print(DictData['column_names'])

['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']


## Question 3:
3.Calculate what the highest and lowest opening prices were for the stock in this period.

In [8]:
#get index of opening prices:
OpeningPriceIndex=DictData['column_names'].index('Open')

#Get All OpeningPrices Over all Data from 2017:
OpeningPrices=[Day[1] for Day in DictData['data']]

MinOpenPrice=min(Price for Price in OpeningPrices if Price != None) #Data contains None values
MaxOpen=max(Price for Price in OpeningPrices if Price != None)

print('The Minimum Opening Price for 2017 was: '+str(MinOpenPrice)+
      '\nThe Maximum Opening Price for 2017 was: '+str(MaxOpen))

The Minimum Opening Price for 2017 was: 34.0
The Maximum Opening Price for 2017 was: 53.11


## Question 4:
4.What was the largest change in any one day (based on High and Low price)?

In [9]:
#get index of Day High/low prices:
DailyHighIndex=DictData['column_names'].index('High')
DailyLowIndex=DictData['column_names'].index('Low')

DailyDiff=[Day[DailyHighIndex]-Day[DailyLowIndex] for Day in DictData['data'] 
           if Day[DailyHighIndex]!=None and Day[DailyLowIndex]!=None]
MaxDailyDiff=max(DailyDiff)
print('The Maximum Daily Difference is: '+str(MaxDailyDiff))

The Maximum Daily Difference is: 2.8100000000000023


## Question 5:
5.What was the largest change between any two days (based on Closing Price)?


In [10]:
#get index of Day Closing prices:
DailyCloseIndex=DictData['column_names'].index('Close')

#Get List of Daily Closing price:
DailyClosingPrice=[Day[DailyCloseIndex] for Day in DictData['data']]

#Get List of differences when none of the Closing Prices is None type

DailyClosingDiffs=[abs(Day1-Day2)for Day1,Day2 in zip(DailyClosingPrice[:-1],DailyClosingPrice[1:]) if Day1!=None and Day2!=None]

MaxDiff=max(DailyClosingDiffs)

print('Biggest Change betweent any two CONSECUTIVE days is: '+str(MaxDiff))


Biggest Change betweent any two CONSECUTIVE days is: 2.559999999999995


## Question 6:
6.What was the average daily trading volume during this year?


In [11]:
#get index of Day Volume:
DailyVolumeIndex=DictData['column_names'].index('Traded Volume')

#Get List of Daily Volume:
DailyVolume=[Day[DailyVolumeIndex] for Day in DictData['data'] if Day[DailyVolumeIndex]!=None]

MeanAnnualVolume=sum(DailyVolume)/len(DailyVolume)

print('Average Daily Volume for 2017 is: '+str(MeanAnnualVolume))


Average Daily Volume for 2017 is: 89124.33725490196


## Question 7:
7.(Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [12]:
def CalculateMedian(List):
    """Calculates the median of a list of numbers"""
    List=sorted(List)
    lenlist=len(List)
    
    if lenlist%2==0:
        return 0.5*(List[(lenlist/2)-1]+List[(lenlist/2)])
    else:
        return List[(lenlist//2)]
    


In [13]:
#get index of Day Volume:
DailyVolumeIndex=DictData['column_names'].index('Traded Volume')

#Get List of Daily Volume:
DailyVolume=[Day[DailyVolumeIndex] for Day in DictData['data'] if Day[DailyVolumeIndex]!=None]

MedianAnnualVolume=CalculateMedian(DailyVolume)
print('The median daily volume for 2017 was: ' +str(MedianAnnualVolume))

The median daily volume for 2017 was: 76286.0
